In [35]:
import os
import re
import numpy as np
import pandas as pd
import nltk
from collections import Counter
from scipy.cluster.hierarchy import dendrogram
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
# Modules from scikit-learn
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

In [36]:
import nltk  
nltk.download('stopword')
nltk.download('punkt')

[nltk_data] Error loading stopword: Package 'stopword' not found in
[nltk_data]     index
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Roy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
fake = pd.read_csv("Dataset/Fake.csv")
true = pd.read_csv("Dataset/True.csv")

# Tokenizing

In [38]:
## Looping through and adding every text to a string for True
trueCorpus = []
fakeCorpus = []
for i in true['text']:
    trueCorpus.append(i) 

for i in fake['text']:
    fakeCorpus.append(i) 

In [39]:
## Tokenizing texts with NLTK
tokenized_True = [word_tokenize(i) for i in trueCorpus]
tokenized_Fake = [word_tokenize(i) for i in fakeCorpus]

In [40]:
print("trueCorpus Length: "+ str(len(tokenized_True)))
print("fakeCorpus Length: "+ str(len(tokenized_Fake)))

trueCorpus Length: 21417
fakeCorpus Length: 23481


In [41]:
## Removing Stopwords from the Corpus
stop_words = set(stopwords.words('english'))
filteredTrue = []
filteredFake = []

for wordList in tokenized_True:
    for word in wordList:
        if word not in stop_words:
            filteredTrue.append(word)

for word in tokenized_True:
    for word in wordList:
        if word not in stop_words:
            filteredFake.append(word)

In [42]:
print(len(filteredTrue))

6177937


In [43]:
print(filteredTrue[0:1000])

['WASHINGTON', '(', 'Reuters', ')', '-', 'The', 'head', 'conservative', 'Republican', 'faction', 'U.S.', 'Congress', ',', 'voted', 'month', 'huge', 'expansion', 'national', 'debt', 'pay', 'tax', 'cuts', ',', 'called', '“', 'fiscal', 'conservative', '”', 'Sunday', 'urged', 'budget', 'restraint', '2018', '.', 'In', 'keeping', 'sharp', 'pivot', 'way', 'among', 'Republicans', ',', 'U.S.', 'Representative', 'Mark', 'Meadows', ',', 'speaking', 'CBS', '’', '“', 'Face', 'Nation', ',', '”', 'drew', 'hard', 'line', 'federal', 'spending', ',', 'lawmakers', 'bracing', 'battle', 'January', '.', 'When', 'return', 'holidays', 'Wednesday', ',', 'lawmakers', 'begin', 'trying', 'pass', 'federal', 'budget', 'fight', 'likely', 'linked', 'issues', ',', 'immigration', 'policy', ',', 'even', 'November', 'congressional', 'election', 'campaigns', 'approach', 'Republicans', 'seek', 'keep', 'control', 'Congress', '.', 'President', 'Donald', 'Trump', 'Republicans', 'want', 'big', 'budget', 'increase', 'military',

In [44]:
print(stop_words)

{'she', 'those', 'he', 'shouldn', 'her', 've', 'its', 'during', 'themselves', 'hers', 'then', "you're", 'i', 'couldn', 'who', 'do', 'own', "should've", "you'll", "hasn't", "you'd", 'not', 'this', 'other', 'into', 'had', 'until', 'any', 'm', 'needn', 'through', 'didn', 'them', 'his', 'by', 'against', "wasn't", "didn't", 'before', 're', 'our', 'all', 'nor', 'both', 'having', 'mustn', 'why', 'with', 'isn', 'to', 'below', "doesn't", 'such', 'it', 'wouldn', 'my', 'we', 'or', 'most', 'weren', 'some', 'aren', 'out', 'don', 'your', 'be', 'being', "don't", 'after', 'than', 'they', 'while', 'no', 'very', 'herself', 'over', 'as', 'mightn', 'yours', 'is', "won't", 'did', 'how', 'doesn', 'me', 'if', 'a', 'further', 'wasn', 'at', 'd', 'o', "mightn't", 'above', 'each', 'up', 'again', "couldn't", 'just', 'few', 'where', 'too', 'these', 'were', 'when', "aren't", 'the', 'but', 'was', 'for', 'yourselves', 'of', 'ours', 'between', 'so', 'more', 'been', 'under', "wouldn't", 'll', 'whom', "you've", "isn't",

In [47]:
##Removing Punctuation 

tempTrue  = ' '.join([str(elem) for elem in filteredTrue])
tokenizer = nltk.RegexpTokenizer(r"\w+")
filteredTrue = tokenizer.tokenize(tempTrue)


tempFalse  = ' '.join([str(elem) for elem in filteredFake])
tokenizer = nltk.RegexpTokenizer(r"\w+")
filteredFake = tokenizer.tokenize(tempFalse)

In [49]:
print(len(filteredFake))
print(len(filteredTrue))

3126882
5291898


# Clustering 

Description Here

# Feature Analysis

Description Here